In [1]:
from google.colab import files
uploaded = files.upload()

Saving dataset_factures_clients.csv to dataset_factures_clients.csv


In [2]:
!pip install wordninja
!pip install pyspellchecker
import pandas as pd
import wordninja
import numpy as np
import re
from spellchecker import SpellChecker

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 541.6/541.6 kB 26.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for wordninja: filename=wordninja-2.0.0-py3-none-any.whl size=541530 sha256=17b1a7f4a44d8b2c718c8b87e86b54785404f3cfa3b9976406eb6f698a7b6663
  Stored in directory: /root/.cache/pip/wheels/e6/66/9c/712044a983337f5d44f90abcd244bd4b8ad28ee64750404b50
Successfully built wordninja
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 95.5 MB/s eta 0:00:00


In [11]:

df = pd.read_csv("dataset_factures_clients.csv")
df3 = df.copy()
## Séparation de texte
def segmenter_avec_separateurs(texte):
    if pd.isna(texte):
        return ""

    # 1. Découper selon les séparateurs : '-', '/', etc.
    morceaux = re.split(r'([-/])', texte)  # on garde les séparateurs dans la liste

    resultat = []
    for morceau in morceaux:
        if morceau in ['-', '/']:  # si c'est un séparateur, on le garde tel quel
            resultat.append(morceau)
        else:
            mots = wordninja.split(morceau)
            resultat.append(" ".join(mots))

    return " ".join(resultat)
df

,ID,Nom Client,Montant (€),Date Facture,Payé ?
0,1,Dupont SARL,1.750,03/03/2023,oui
1,2,Durand SA,1 000,2023-03-23,Oui
2,3,durand sa,1 000,06-06-2023,NaN
3,4,NaN,2500,12/06/2023,NON
4,5,Martin & Fils,deux mille,05/03/2023,NON
...,...,...,...,...,...
295,296,Entreprise X,2500,2023-05-07,Non précisé
296,297,dupont sarl,deux mille,01-04-2023,NON
297,298,SARL Jaune,1.750,28/03/2023,Oui
298,299,entreprise x,2500,05-01-2023,NaN


In [17]:
## Tableau avec inconnu
df3.columns = df3.columns.str.strip()
df3 = df3.applymap(lambda x : str(x).strip() if pd.notnull(x) else x)
df3["Payé ?"] = df3["Payé ?"].str.capitalize()
df3["Payé ?"] = df3["Payé ?"].replace("Non précisé", np.nan, regex = False)
df3["Payé ?"].replace(["nan"], "Non",inplace = True)
df3.replace(["durand sa", "MARTIN & FILS", "ENTREPRISE X", "sarl jaune", "entreprise x", "dupont sarl", "DURAND SA", "martin & fils","DUPONT SARL","SARL JAUNE"],
 ["Durand SA", "Martin & Fils", "Entreprise X","SARL Jaune", "Entreprise X", "Dupont SARL", "Durand SA", "Martin & Fils", "Dupont SARL","SARL Jaune"], inplace = True)
df3["Montant (€)"].unique()
df3.replace(["1 000", "1.750", "deux mille", "1800€", "2,300"], ["1000", "1750", "2000", "1800", "2300"], inplace = True)

df3["Payé ?"]. unique()
df3 = df3.dropna(axis=0, how='all')
df3 = df3[df3.isnull().sum(axis=1) < 2]
df3[df3["Nom Client"].isnull()]
df3.fillna("Inconnu", inplace = True)
df3


/tmp/ipython-input-17-2112707828.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df3 = df3.applymap(lambda x : str(x).strip() if pd.notnull(x) else x)
/tmp/ipython-input-17-2112707828.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df3["Payé ?"].replace(["nan"], "Non",inplace = True)


,ID,Nom Client,Montant (€),Date Facture,Payé ?
0,1,Dupont SARL,1750,03/03/2023,Oui
1,2,Durand SA,1000,23/03/2023,Oui
2,3,Durand SA,1000,06/06/2023,Inconnu
3,4,Inconnu,2500,12/06/2023,Non
4,5,Martin & Fils,2000,05/03/2023,Non
...,...,...,...,...,...
295,296,Entreprise X,2500,05/07/2023,Inconnu
296,297,Dupont SARL,2000,01/04/2023,Non
297,298,SARL Jaune,1750,28/03/2023,Oui
298,299,Entreprise X,2500,05/01/2023,Inconnu


In [18]:
## Changement format des dates
df3["Date Facture"] = df3["Date Facture"].astype(str).str.strip().str.replace("-", "/", regex=False)
def parser_date_personnalisé(date_str):
    if pd.isnull(date_str):
        return np.nan
    date_str = str(date_str).strip()

    # Cas 1 : format YYYY-MM-DD
    if re.fullmatch(r"\d{4}-\d{2}-\d{2}", date_str):
        try:
            return pd.to_datetime(date_str, format="%Y-%m-%d")
        except:
            return np.nan

    # Cas 2 : format DD-MM-YYYY
    if re.fullmatch(r"\d{2}-\d{2}-\d{4}", date_str):
        try:
            return pd.to_datetime(date_str, format="%d-%m-%Y")
        except:
            return np.nan

    # Cas 3 : format DD/MM/YYYY
    if re.fullmatch(r"\d{2}/\d{2}/\d{4}", date_str):
        try:
            return pd.to_datetime(date_str, format="%d/%m/%Y")
        except:
            return np.nan

    # Autres cas : on essaie quand même
    try:
        return pd.to_datetime(date_str, dayfirst=True)
    except:
        return np.nan

# Appliquer la fonction
df3["Date Facture"] = df3["Date Facture"].apply(parser_date_personnalisé)

# Format final en DD/MM/YYYY
df3["Date Facture"] = df3["Date Facture"].dt.strftime("%d/%m/%Y")
df3

,ID,Nom Client,Montant (€),Date Facture,Payé ?
0,1,Dupont SARL,1750,03/03/2023,Oui
1,2,Durand SA,1000,23/03/2023,Oui
2,3,Durand SA,1000,06/06/2023,Inconnu
3,4,Inconnu,2500,12/06/2023,Non
4,5,Martin & Fils,2000,05/03/2023,Non
...,...,...,...,...,...
295,296,Entreprise X,2500,05/07/2023,Inconnu
296,297,Dupont SARL,2000,01/04/2023,Non
297,298,SARL Jaune,1750,28/03/2023,Oui
298,299,Entreprise X,2500,05/01/2023,Inconnu


In [19]:
aligne = df3.style.set_properties(**{'text-align': 'left'})
aligne

,ID,Nom Client,Montant (€),Date Facture,Payé ?
0,1,Dupont SARL,1750,03/03/2023,Oui
1,2,Durand SA,1000,23/03/2023,Oui
2,3,Durand SA,1000,06/06/2023,Inconnu
3,4,Inconnu,2500,12/06/2023,Non
4,5,Martin & Fils,2000,05/03/2023,Non
5,6,Martin & Fils,Inconnu,12/02/2023,Oui
6,7,Entreprise X,Inconnu,02/03/2023,Oui
7,8,Dupont SARL,1750,13/01/2023,Oui
8,9,Durand SA,1000,09/01/2023,Oui
10,11,Martin & Fils,1750,04/10/2023,Non


In [20]:
from google.colab import files
aligne.to_excel("Factures_nettoyées.xlsx", index = False)
files.download("Factures_nettoyées.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>